In [73]:
# pip install -U sentence-transformers fastapi uvicorn
# !pip install docling

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from IPython.display import  clear_output
import time
import PyPDF2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

# Configure 4-bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4",  # Normalized float 4-bit (recommended)
    bnb_4bit_compute_dtype=torch.float16,  
    bnb_4bit_use_double_quant=True  # Improves performance by applying second quantization
)

model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    quantization_config=bnb_config,
    # torch_dtype=torch.bfloat16,
    device_map=device,
)

tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = 128001

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model.get_memory_footprint()/(1024*1024*1024)# Gbs

2.046719789505005

# Step 1: Reading PDF Docs - Using PyPDF

In [3]:
pdf_path = 'bhagavad-gita-in-english-source-file.pdf'
pdf_reader = PyPDF2.PdfReader(pdf_path)
# Get total number of pages
num_pages = len(pdf_reader.pages)
print(f"Processing PDF with {num_pages} pages...")

full_text = []

for page_num in range(4,num_pages-1):
    page = pdf_reader.pages[page_num]
    text = page.extract_text()
    full_text.append(text)

full_text = '\n'.join(full_text)

print(full_text)


Processing PDF with 53 pages...
 
  
 
BHAGAVAD -GITA in ENGLISH  
Author: Sage Veda Vy asa 
Translat ed in English : Ramananda Prasad , Ph.D.  
Language Editor s: Needed  
Contact: rprasad@gita -society.com  
***** 
“Let noble thoughts come to us from everywhere”    
(The Vedas)  
 
INTRODUCTION  
The Bhagavad -Gita is a doctrine of universal truth  and a book 
of moral and spiritual growth . Its message is sublime and non -sec-
tarian . It deals with the most sacred metaphysi cal science. It im-
parts the knowledge of the Self and answers two universal ques-
tions: Who am I, and how can I lead a happy and peaceful life in 
this wor d full of dualities  and dilemmas ?  
It's a timeless book of wisdom  that inspired Thoreau, Emerson, 
Einstein, Oppenheimer, Gandhi and many others. The Bhagavad -
Gita teaches us how  to equip ourselves  for the battle of life. A re-
peated study with faith purifies our psyche and guides us to face 
the challenges of modern livin g leading to inner peace

# Step 1: Reading PDF Docs - Using Docling

In [4]:
from docling.document_converter import DocumentConverter
pdf_path = 'bhagavad-gita-in-english-source-file.pdf'
converter = DocumentConverter()


In [5]:
result = converter.convert(pdf_path,page_range=[5,51])
full_text = result.document.export_to_text()
print(full_text)

c:\Users\User\miniconda3\envs\torch_env\Lib\site-packages\torch\utils\cpp_extension.py:382: UserWarning: Error checking compiler version for cl: [WinError 2] The system cannot find the file specified
  warnings.warn(f'Error checking compiler version for {compiler}: {error}')
Could not load the custom kernel for multi-scale deformable attention: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-scale deformable attention: DLL load failed while importing MultiScaleDeformableAttention: The specified module could not be found.
Could not load the custom kernel for multi-s

## BHAGAVAD-GITA in ENGLISH

## Author: Sage Veda Vyasa

Translated in English: Ramananda Prasad, Ph.D. Language Editors: Needed Contact: rprasad@gita-society.com

*****

'Let noble thoughts come to us from everywhere' (The Vedas)

## INTRODUCTION

The Bhagavad-Gita is a doctrine of universal truth and a book of moral and spiritual growth. Its message is sublime and non-sectarian. It deals with the most sacred metaphysical science. It imparts the knowledge of the Self and answers two universal questions: Who am I, and how can I lead a happy and peaceful life in this word full of dualities and dilemmas?

It's a timeless book of wisdom that inspired Thoreau, Emerson, Einstein, Oppenheimer, Gandhi and many others. The BhagavadGita teaches us how to equip ourselves for the battle of life. A repeated study with faith purifies our psyche and guides us to face the challenges of modern living leading to inner peace and happiness.

Gita teaches the spiritual science of Self-realization (SR) bas

# Step 2- Chuncking Method

In [7]:
from langchain.text_splitter import TokenTextSplitter,RecursiveCharacterTextSplitter,NLTKTextSplitter,SpacyTextSplitter,MarkdownHeaderTextSplitter


In [8]:
splitter = TokenTextSplitter(chunk_size=100,chunk_overlap=0)
chunk_list = splitter.split_text(full_text)

print(f'Number of chunks: {len(chunk_list)}')
print('*'*50)

print(chunk_list[0])
print('*'*50)
print(chunk_list[1])
print('*'*50)
print(chunk_list[2])

Number of chunks: 323
**************************************************
## BHAGAVAD-GITA in ENGLISH

## Author: Sage Veda Vyasa

Translated in English: Ramananda Prasad, Ph.D. Language Editors: Needed Contact: rprasad@gita-society.com

*****

'Let noble thoughts come to us from everywhere' (The Vedas)

## INTRODUCTION

The Bhagavad-Gita is
**************************************************
 a doctrine of universal truth and a book of moral and spiritual growth. Its message is sublime and non-sectarian. It deals with the most sacred metaphysical science. It imparts the knowledge of the Self and answers two universal questions: Who am I, and how can I lead a happy and peaceful life in this word full of dualities and dilemmas?

It's a timeless book of wisdom that inspired Thoreau, Emerson, Einstein, Oppenheimer, Gandhi and many others
**************************************************
. The BhagavadGita teaches us how to equip ourselves for the battle of life. A repeated study with faith

In [10]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=0)
chunk_list = splitter.split_text(full_text)

print(f'Number of chunks: {len(chunk_list)}')
print('*'*50)

print(chunk_list[0])
print('*'*50)
print(chunk_list[1])
print('*'*50)
print(chunk_list[2])

Number of chunks: 402
**************************************************
## BHAGAVAD-GITA in ENGLISH

## Author: Sage Veda Vyasa

Translated in English: Ramananda Prasad, Ph.D. Language Editors: Needed Contact: rprasad@gita-society.com

*****

'Let noble thoughts come to us from everywhere' (The Vedas)

## INTRODUCTION
**************************************************
The Bhagavad-Gita is a doctrine of universal truth and a book of moral and spiritual growth. Its message is sublime and non-sectarian. It deals with the most sacred metaphysical science. It imparts the knowledge of the Self and answers two universal questions: Who am I, and how can I lead a happy and peaceful life in this word full of dualities and dilemmas?
**************************************************
It's a timeless book of wisdom that inspired Thoreau, Emerson, Einstein, Oppenheimer, Gandhi and many others. The BhagavadGita teaches us how to equip ourselves for the battle of life. A repeated study with faith pur

In [21]:
splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("##", "Header_2"),
        ],
        return_each_line=False
    )

chunk_list = splitter.split_text(full_text)

print(f'Number of chunks: {len(chunk_list)}')
print('chunk_1','*'*50)
print(chunk_list[0].page_content)
print('chunk_2','*'*50)
print(chunk_list[1].page_content)
print('chunk_3','*'*50)
print(chunk_list[2].page_content)

Number of chunks: 105
chunk_1 **************************************************
Translated in English: Ramananda Prasad, Ph.D. Language Editors: Needed Contact: rprasad@gita-society.com  
*****  
'Let noble thoughts come to us from everywhere' (The Vedas)
chunk_2 **************************************************
The Bhagavad-Gita is a doctrine of universal truth and a book of moral and spiritual growth. Its message is sublime and non-sectarian. It deals with the most sacred metaphysical science. It imparts the knowledge of the Self and answers two universal questions: Who am I, and how can I lead a happy and peaceful life in this word full of dualities and dilemmas?  
It's a timeless book of wisdom that inspired Thoreau, Emerson, Einstein, Oppenheimer, Gandhi and many others. The BhagavadGita teaches us how to equip ourselves for the battle of life. A repeated study with faith purifies our psyche and guides us to face the challenges of modern living leading to inner peace and happine

In [22]:
cleaned_text = [text.page_content for text in chunk_list]

In [23]:
import pickle

# Open a file in write-binary mode and save the list
with open("cleaned_text.pkl", "wb") as f:
    pickle.dump(cleaned_text, f)
    
with open("cleaned_text.pkl", "rb") as f:
    cleaned_text = pickle.load(f)

print("Loaded list:", cleaned_text)

Loaded list: ["Translated in English: Ramananda Prasad, Ph.D. Language Editors: Needed Contact: rprasad@gita-society.com  \n*****  \n'Let noble thoughts come to us from everywhere' (The Vedas)", "The Bhagavad-Gita is a doctrine of universal truth and a book of moral and spiritual growth. Its message is sublime and non-sectarian. It deals with the most sacred metaphysical science. It imparts the knowledge of the Self and answers two universal questions: Who am I, and how can I lead a happy and peaceful life in this word full of dualities and dilemmas?  \nIt's a timeless book of wisdom that inspired Thoreau, Emerson, Einstein, Oppenheimer, Gandhi and many others. The BhagavadGita teaches us how to equip ourselves for the battle of life. A repeated study with faith purifies our psyche and guides us to face the challenges of modern living leading to inner peace and happiness.  \nGita teaches the spiritual science of Self-realization (SR) based on the essence of Upanishads and Vedanta. The 

In [24]:
print(cleaned_text[11])

The Supreme Lord said: When one is completely free from all desires of the mind and is satisfied with the Eternal Being (God) by the joy of Eternal Being, then one is called an enlightened person, O Arjuna. (2.55) A person whose mind is unperturbed by sorrow, who does not crave pleasures - and who is completely free from attachment,  fear,  and  anger-  is  called  a  sage  of  steady  mind. (2.56) Those who are not attached to anything, who are neither elated by getting desired results, nor troubled by undesired results, their  intellect  is  considered  steady.  (2.57)  When  one  can  completely withdraw the senses from its sense objects- as a tortoise withdraws its limbs into the shell for protection from dangers- then the intellect of such a person is considered steady. (2.58) The desire for sensual pleasures fades away if one abstains from sense enjoyment, but the craving for sense enjoyment remains. The craving also disappears from one who has known the Supreme Being. (2.59)


# Creating Vector database

In [25]:
import numpy as np
import umap
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
embedding_model = SentenceTransformer("all-MiniLM-L12-v2")#intfloat/e5-large-v2, all-MiniLM-L12-v2,all-mpnet-base-v2

In [26]:
embeddings = embedding_model.encode(cleaned_text, convert_to_tensor=True, normalize_embeddings=True)
print(embeddings.shape)
torch.save(embeddings, "vector_embeddings.pt")

torch.Size([105, 384])


# The Retriver

In [27]:
# Is there an afterlife? What happens when we die?
# I often feel lost in life. How do I find my true purpose?
# Why does God allow suffering and evil in the world?
# What is the role of prayer, and does it really work?
#'what is the meaning of life?'
query = 'Is there an afterlife? What happens when we die?'

## QUERY re-writing

In [4]:
def rewrite_query(query):

    conversation = [{"role": "system", "content": '''You are a helpful assistant specialized in spiritual and philosophical teachings of the Bhagavad Gita.

    Your task is to take a user’s natural language question and rewrite it in a way that makes it easier to retrieve the most relevant teachings or verses from the Bhagavad Gita. The rewritten query should preserve the original meaning, but include spiritual keywords and core themes such as: dharma (duty), karma (action), atman (soul), detachment, self-control, anger, devotion, renunciation, knowledge, and meditation.

    The output should be a single, concise, clear query that captures the essence of the user’s question using language aligned with the teachings of the Gita.

    Examples:

    Input: "I struggle to control my anger. What can I do to remain peaceful?"
    Output: "Teachings on controlling anger and attaining inner peace through detachment and self-discipline."

    Input: "How can I find my purpose in life?"
    Output: "Insights on discovering one's dharma and fulfilling it with devotion and detachment from results."

    Input: "Why do bad things happen even when I do good?"
    Output: "Understanding karma and the importance of acting righteously without attachment to outcomes."

    ALWAYS start your response directly with processed text and NO ACKNOWLEDGEMENTS about my questions ok? Now rewrite the following user query accordingly.'''},
                    
    {"role": "user", "content": f'''{query}'''},

    ]
    # 
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)


    with torch.no_grad():
        output = model.generate(
            **inputs,
            do_sample=True,
            max_new_tokens=256
        )


    rewritten_query = tokenizer.decode(output[0][len(inputs.input_ids[0])+3:], skip_special_tokens=True)
    
    return rewritten_query.strip()

rewritten_query = rewrite_query(query)

print(f"Original Query: {query}")
print(f"Rewritten Query: {rewritten_query}")

NameError: name 'query' is not defined

## Using Bi-encoder to retrive ralevent Chunks from the Vector DataBase

In [30]:
query_encoded = embedding_model.encode([rewritten_query], convert_to_tensor=True)
similarities = embedding_model.similarity(query_encoded, embeddings)
scores, top_k_indices = torch.topk(similarities[0], k=10)
print(scores, top_k_indices)

# Step 6: Display results
print("Top 5 similar chunks:")
for idx in top_k_indices:
    print(f"Score: {similarities[0][idx]:.4f} | Chunk-{idx}: {cleaned_text[idx]} \n")
    print('*'*100)

tensor([0.6128, 0.5300, 0.5231, 0.5181, 0.5144, 0.5039, 0.5031, 0.4950, 0.4749,
        0.4609], device='cuda:0') tensor([76,  9, 93, 22, 38, 40, 49, 27, 31, 18], device='cuda:0')
Top 5 similar chunks:
Score: 0.6128 | Chunk-76: The eternal individual soul in the body of living beings is, indeed, My integral part. It associates with the six sensory faculties of perception - including the mind - and activates them. (15.07) Just as the air takes aroma away from the flower; similarly, the individual soul takes the six sensory faculties from the physical body it casts off during death to the new physical body it acquires in reincarnation by the power of Karma. (15.08) The living entity (Jiva)  
enjoys  sense  pleasures  using  six  sensory  faculties  of  hearing, touch, sight, taste, smell, and mind. The ignorant cannot perceive Jiva departing from the body, nor staying in the body and enjoying sense pleasures by associating with the modes of material Nature. But those who have the eye of 

## Using cross encoder to Rerank the CHUNKS according to their similarity with the QUERY

In [31]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

In [32]:
candidate_pairs = [(rewritten_query, cleaned_text[i]) for i in top_k_indices]
# Step 4: Use cross-encoder to rerank
cross_scores = cross_encoder.predict(candidate_pairs)

# Step 5: Sort by descending scores
reranked = sorted(zip(cross_scores, scores, top_k_indices), key=lambda x: x[0], reverse=True)

# Step 6: Display results
print("🔝 Reranked Chunks:")
for rerank_score,score, idx in reranked:
    print(f"Cross-Encoder Score: {rerank_score:.4f} | Biencoder score: {score:.4f} | Chunk-{idx}: {cleaned_text[idx]}")
    print('*'*100)

🔝 Reranked Chunks:
Cross-Encoder Score: -0.0039 | Biencoder score: 0.6128 | Chunk-76: The eternal individual soul in the body of living beings is, indeed, My integral part. It associates with the six sensory faculties of perception - including the mind - and activates them. (15.07) Just as the air takes aroma away from the flower; similarly, the individual soul takes the six sensory faculties from the physical body it casts off during death to the new physical body it acquires in reincarnation by the power of Karma. (15.08) The living entity (Jiva)  
enjoys  sense  pleasures  using  six  sensory  faculties  of  hearing, touch, sight, taste, smell, and mind. The ignorant cannot perceive Jiva departing from the body, nor staying in the body and enjoying sense pleasures by associating with the modes of material Nature. But those who have the eye of Self-knowledge can see it. (15.0910) The yogis, striving for perfection, behold the living entity (Jiva) abiding in their inner psyche as cons

In [33]:
# Create the context with top 5 chunks
k_top = 5
CONTEXT_TEXT = '\n\n'.join([cleaned_text[idx] for _,_,idx in reranked[:k_top]])

print(CONTEXT_TEXT)

The eternal individual soul in the body of living beings is, indeed, My integral part. It associates with the six sensory faculties of perception - including the mind - and activates them. (15.07) Just as the air takes aroma away from the flower; similarly, the individual soul takes the six sensory faculties from the physical body it casts off during death to the new physical body it acquires in reincarnation by the power of Karma. (15.08) The living entity (Jiva)  
enjoys  sense  pleasures  using  six  sensory  faculties  of  hearing, touch, sight, taste, smell, and mind. The ignorant cannot perceive Jiva departing from the body, nor staying in the body and enjoying sense pleasures by associating with the modes of material Nature. But those who have the eye of Self-knowledge can see it. (15.0910) The yogis, striving for perfection, behold the living entity (Jiva) abiding in their inner psyche as consciousness, but the ignorant and those whose inner psyche is not pure, even though stri

# Passing the QUERY and CONTEXT to LLM

In [34]:
def gita_RAG(query, context):
    rag_prompt = [
        {
            "role": "system",
            "content": (
                "You are a knowledgeable assistant well-versed in the Bhagavad Gita. "
                "Use the provided context to answer the user's query faithfully. "
                "If the answer is not in the context, say you don't know. "
                "Your goal is to provide clear, concise, and spiritually aligned answers in simple english."
                "The response should not be too long, at max 2-3 sentences. "
            )
        },
        {
            "role": "user",
            "content": (
                f"Context:\n{context}\n\n"
                f"Question:\n{query}\n\n"
                f"Answer:"
            )
        }
    ]
    # 
    prompt = tokenizer.apply_chat_template(rag_prompt, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)


    with torch.no_grad():
        output = model.generate(
            **inputs,
            do_sample=True,
            max_new_tokens=256
        )


    processed_text = tokenizer.decode(output[0][len(inputs.input_ids[0])+3:], skip_special_tokens=True)

    return processed_text.strip()



processed_text = gita_RAG(query, CONTEXT_TEXT)

print(processed_text)

According to the Bhagavad Gita, when we die, our individual souls (Jiva) depart from the physical body, but the Supreme Being or Eternal Being (Paramatman) remains. They continue to exist in the higher realms of existence, beyond the physical world. This is described as the world of Brahma, the creator, and beyond.


# Putting it all together

In [6]:
from sentence_transformers import SentenceTransformer,CrossEncoder
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch


device = 'cuda' if torch.cuda.is_available() else 'cpu'

DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

# Configure 4-bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4",  # Normalized float 4-bit (recommended)
    bnb_4bit_compute_dtype=torch.float16,  
    bnb_4bit_use_double_quant=True  # Improves performance by applying second quantization
)

model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    quantization_config=bnb_config,
    # torch_dtype=torch.bfloat16,
    device_map=device,
)

tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = 128001

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
import pickle

vector_db = torch.load("vector_embeddings.pt").to(device)
with open("cleaned_text.pkl", "rb") as f:
    cleaned_text = pickle.load(f)
    

embedding_model = SentenceTransformer("all-MiniLM-L12-v2")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rewrite_query(query):

    conversation = [{"role": "system", "content": '''You are a helpful assistant specialized in spiritual and philosophical teachings of the Bhagavad Gita.

    Your task is to take a user’s natural language question and rewrite it in a way that makes it easier to retrieve the most relevant teachings or verses from the Bhagavad Gita. The rewritten query should preserve the original meaning, but include spiritual keywords and core themes such as: dharma (duty), karma (action), atman (soul), detachment, self-control, anger, devotion, renunciation, knowledge, and meditation.

    The output should be a single, concise, clear query that captures the essence of the user’s question using language aligned with the teachings of the Gita.

    Examples:

    Input: "I struggle to control my anger. What can I do to remain peaceful?"
    Output: "Teachings on controlling anger and attaining inner peace through detachment and self-discipline."

    Input: "How can I find my purpose in life?"
    Output: "Insights on discovering one's dharma and fulfilling it with devotion and detachment from results."

    Input: "Why do bad things happen even when I do good?"
    Output: "Understanding karma and the importance of acting righteously without attachment to outcomes."

    ALWAYS start your response directly with processed text and NO ACKNOWLEDGEMENTS about my questions ok? Now rewrite the following user query accordingly.'''},
                    
    {"role": "user", "content": f'''{query}'''},

    ]
    # 
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)


    with torch.no_grad():
        output = model.generate(
            **inputs,
            do_sample=True,
            max_new_tokens=256
        )


    rewritten_query = tokenizer.decode(output[0][len(inputs.input_ids[0])+3:], skip_special_tokens=True)
    
    return rewritten_query.strip()


def gita_RAG(query, context):
    rag_prompt = [
        {
            "role": "system",
            "content": (
                "You are a knowledgeable assistant well-versed in the Bhagavad Gita. "
                "Use the provided context to answer the user's query faithfully. "
                "If the answer is not in the context, say you don't know. "
                "Your goal is to provide clear, concise, and spiritually aligned answers in simple english."
                "Quote the text you have used from the Gita context."
            )
        },
        {
            "role": "user",
            "content": (
                f"Context:\n{context}\n\n"
                f"Question:\n{query}\n\n"
                f"Answer:"
            )
        }
    ]
    # 
    prompt = tokenizer.apply_chat_template(rag_prompt, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)


    with torch.no_grad():
        output = model.generate(
            **inputs,
            do_sample=False,
            max_new_tokens=256
        )


    processed_text = tokenizer.decode(output[0][len(inputs.input_ids[0])+3:], skip_special_tokens=True)

    return processed_text.strip()

C:\Users\User\AppData\Local\Temp\ipykernel_15568\3776774921.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vector_db = torch.load("vector_embeddings.pt").to(device)


In [14]:
query = 'I struggle to control my anger. What can I do to remain peaceful?'
rewritten_query = rewrite_query(query)

In [15]:
# Step 1: Encode the rewritten query using the bi-encoder (embedding model)
# This transforms the text into a dense vector representation
query_encoded = embedding_model.encode([rewritten_query], convert_to_tensor=True)

# Step 2: Compute cosine similarity between the query and all stored document embeddings
similarities = embedding_model.similarity(query_encoded, vector_db)

# Step 3: Select the top-k most similar chunks based on embedding similarity (bi-encoder stage)
# similarities[0] gives similarity scores for the first (and only) query
scores, top_k_indices = torch.topk(similarities[0], k=10)

# Step 4: Prepare (query, chunk) pairs for reranking using cross-encoder
# This enables more accurate relevance scoring by modeling token-level interactions
candidate_pairs = [(rewritten_query, cleaned_text[i]) for i in top_k_indices]

# Step 5: Use cross-encoder to compute a more accurate relevance score for each pair
cross_scores = cross_encoder.predict(candidate_pairs)

# Step 6: Rerank the chunks by cross-encoder score (highest score = most relevant)
# Each item is a tuple: (cross_score, bi_score, index)
# You retain the original index so you can fetch the corresponding chunk later
reranked = sorted(zip(cross_scores, scores, top_k_indices), key=lambda x: x[0], reverse=True)

# Step 7: Choose the top-k reranked chunks to be used as context for LLM
k_top = 5
CONTEXT_TEXT = '\n\n'.join([cleaned_text[idx] for _, _, idx in reranked[:k_top]])


In [16]:
print(CONTEXT_TEXT)

A disciplined person, enjoying sense objects with senses that are under control and free from likes and dislikes, attains tranquility. (2.64) All sorrows are destroyed upon attainment of tranquility. The intellect of such a tranquil person soon becomes completely steady and united with the Source. (2.65) There is neither Selfknowledge nor Self-perception to those who are not united with the Eternal Being. Without Self-perception there is no peace, and without peace there can be no happiness. (2.66)  
The mind, when controlled by roving senses, steals away the intellect as a storm takes away a boat on the sea from its destination - the spiritual shore. (2.67) Therefore, O Arjuna, one's intellect becomes steady when the senses are completely withdrawn from its sense objects. (2.68) A yogi, the person of self-restraint, remains wakeful when it is night for all others. It is night for a yogi who sees when all others are wakeful. (2.69)  
NOTE: What is considered real by a yogi is of no val

In [17]:
gita_response = gita_RAG(query, CONTEXT_TEXT)

print(gita_response.strip())

c:\Users\User\miniconda3\envs\torch_env\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\User\miniconda3\envs\torch_env\Lib\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


According to the Bhagavad Gita (2.67), "The mind, when controlled by roving senses, steals away the intellect as a storm takes away a boat on the sea from its destination - the spiritual shore."

To remain peaceful, you should strive to control your senses and bring them under your control. This will help your mind to focus on the spiritual shore, and you will attain tranquility.

As the Bhagavad Gita states in 2.68, "Therefore, O Arjuna, one's intellect becomes steady when the senses are completely withdrawn from their sense objects."

To achieve this, you can practice self-restraint and detachment from worldly desires and attachments. This will help you to cultivate a sense of inner peace and tranquility, even in the midst of challenging situations.

Additionally, as the Bhagavad Gita says in 2.61, "One should fix one's mind on Me with loving contemplation after bringing the senses under control. One's intellect becomes steady when one's senses are under complete control."

By cultiv